# Language models with Keras

Keras does not afford language models out of the box.
Instead, we will build them ourselves using neural networks.

In [5]:
# suppose we have already segmented sentences
sentences = [
    "un ordianteur peut vous aider",
    "il veut vous aider",
    "il veut un ordinateur",
    "il peut nager"
]

sentences

['un ordianteur peut vous aider',
 'il veut vous aider',
 'il veut un ordinateur',
 'il peut nager']

## I. Simple FeedForward 3-gram model

In [14]:
N = 3

# Padding the sentences
# we can use NLTK, but I will showcase how to do it just using Keras
# in case you didn't install NLTK
sentencesPad = []
for sentence in sentences:
    new_sentence = sentence
    for i in range(N-1):
        new_sentence = "<s> " + new_sentence + " </s>"
    sentencesPad.append(new_sentence)

sentencesPad

['<s> <s> un ordianteur peut vous aider </s> </s>',
 '<s> <s> il veut vous aider </s> </s>',
 '<s> <s> il veut un ordinateur </s> </s>',
 '<s> <s> il peut nager </s> </s>']

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer

# We can use NLTK vocab to simplify this
tokenizer = Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sentencesPad)
# 10 tokens + UNK
total_words = len(tokenizer.word_index)

sentencesOrdinal = []
for line in sentencesPad:
    token_list = tokenizer.texts_to_sequences([line])[0]
    sentencesOrdinal.append(token_list)

sentencesOrdinal

[[2, 2, 4, 9, 5, 6, 7, 2, 2],
 [2, 2, 3, 8, 6, 7, 2, 2],
 [2, 2, 3, 8, 4, 10, 2, 2],
 [2, 2, 3, 5, 11, 2, 2]]

In [37]:
# create ngrams 
# we can use NLTK, but I will showcase how to do it just using Keras
# in case you didn't install NLTK

# the inputs are two consecutive words
# So, we will have a two dimentinal array
X = []
# the outputs are the third word
# So, we will have a one dimentional array
Y = []

for sentence in sentencesOrdinal:
    slen = len(sentence)
    for i in range(slen-N):
        Xi = []
        for j in range(N-1):
            Xi.append(i+j)
        X.append(Xi)
        Y.append(i+N-1)

X, Y

([[0, 1],
  [1, 2],
  [2, 3],
  [3, 4],
  [4, 5],
  [5, 6],
  [0, 1],
  [1, 2],
  [2, 3],
  [3, 4],
  [4, 5],
  [0, 1],
  [1, 2],
  [2, 3],
  [3, 4],
  [4, 5],
  [0, 1],
  [1, 2],
  [2, 3],
  [3, 4]],
 [2, 3, 4, 5, 6, 7, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5])

In [38]:
from tensorflow.keras.utils import to_categorical
import numpy as np

Ybin = to_categorical(Y, num_classes=total_words)
Xbin = np.array(to_categorical(X, num_classes=total_words))
input_len = Xbin.shape[1] * Xbin.shape[2]
Xbin = Xbin.reshape(Xbin.shape[0], input_len)

Xbin[0,:], Ybin[0]

(array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

### I.1. Without embedding

In [39]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
ff_model = Sequential()
ff_model.add(Dense(10, input_dim=input_len, activation='relu'))
ff_model.add(Dense(total_words, activation='softmax'))
ff_model.compile(loss='categorical_crossentropy', optimizer='adam')
ff_model.fit(Xbin, Ybin, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 253ms/step - loss: 2.5003
Epoch 2/100
1/1 [==============================] - 0s 4ms/step - loss: 2.4934
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 2.4866
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4798
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 2.4730
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 2.4662
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 2.4595
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 2.4527
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4460
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 2.4393
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 2.4326
Epoch 12/100
1/1 [==============================] - 0s 6ms/step - loss: 2.4259
Epoch 13/100
1/1 [==============================] - 0s 5ms

In [54]:
# the function which takes N-1 words separated by space
# and returns the Nth most probable word
def estimate(words):
    Xp = tokenizer.texts_to_sequences([words])[0]
    Xp = to_categorical(Xp, num_classes=total_words)
    Xp = np.array([(np.array(Xp)).flatten()])
    prob = ff_model.predict(Xp)
    i = prob.argmax()
    return tokenizer.sequences_to_texts([[i]])[0]
    
estimate('peut aider')

'vous'

### I.2. With embedding

In [61]:
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Sequential
ff_model_emb = Sequential()
ff_model_emb.add(Embedding(total_words, 10, input_length=N-1))
ff_model_emb.add(Flatten())
ff_model_emb.add(Dense(total_words, activation='softmax'))
ff_model_emb.compile(loss='categorical_crossentropy', optimizer='adam')
#Here we use X instead of Xbin
ff_model_emb.fit(np.array(X), Ybin, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 477ms/step - loss: 2.4217
Epoch 2/100
1/1 [==============================] - 0s 10ms/step - loss: 2.4173
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4129
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4085
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4041
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 2.3997
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 2.3953
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 2.3909
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 2.3865
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 2.3821
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 2.3777
Epoch 12/100
1/1 [==============================] - 0s 3ms/step - loss: 2.3733
Epoch 13/100
1/1 [==============================] - 0s 6ms

In [62]:
# the function which takes N-1 words separated by space
# and returns the Nth most probable word
def estimate(words):
    Xp = tokenizer.texts_to_sequences([words])[0]
    Xp = np.array([Xp])
    prob = ff_model_emb.predict(Xp)
    i = prob.argmax()
    return tokenizer.sequences_to_texts([[i]])[0]
    
estimate('peut aider')

'vous'